# Import Libraries

In [1]:
!pip install -q protobuf==3.19.6
!pip install -q tensorflow==2.11.1 --no-deps
!pip install -q tensorflow-recommenders=='v0.7.3' --no-deps
!pip install -q tensorflow-datasets==3.2.0 --no-deps
!pip install -q tensorflow-metadata==0.22.2 --no-deps
!pip install -q scann
!pip install -q dill==0.3.6

!pip install "uvicorn[standard]>=0.12.0,<0.14.0" fastapi~=0.63 -q
!pip install "google-cloud-storage>=1.26.0,<2.0.0dev"
!pip install google-cloud-aiplatform "shapely<2"
!pip uninstall shapely-2.0.1 -y

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.19.6 which is incompatible.
tensorflow-datasets 4.9.2 requires protobuf>=3.20, but you have protobuf 3.19.6 which is incompatible.
tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 3.19.6 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.3/321.3 kB 30.2 MB/s eta 0:00:00
  Attempting uninstall: shapely
    Found existing installation: shapely 2.0.1
    Uninstalling shapely-2.0.1:
      Successfully uninstalled shapely-2.0.1


In [1]:
import os
import pprint
import tempfile

from typing import Dict, Text

import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

# Data Preparation

In [2]:
# Load ecommerce dataset
df_ecommerce = pd.read_csv('ecommerce.csv') # Contains transactions
df_products = pd.read_csv('products.csv')   # Contains products

In [3]:
# Convert dataframe to tensors
ds_ecommerce = tf.data.Dataset.from_tensor_slices(dict(df_ecommerce))
ds_products = tf.data.Dataset.from_tensor_slices(dict(df_products))

In [4]:
ds_ecommerce = tf.data.Dataset.from_tensor_slices(dict(df_ecommerce))
# Select fields
ds_ecommerce = ds_ecommerce.map(lambda x: {
    'user_id': tf.strings.as_string(x['user_id']),
    'product_id': tf.strings.as_string(x['product_id']),
    'product_name': x['product_name'],
    'product_description': x['product_description'],
    'age': x['age'],
    'search_query': x['search_query']
})

# ds_products = tf.data.Dataset.from_tensor_slices(dict(df_products))
# ds_products = ds_products.batch(32).map(lambda x: tf.strings.as_string(x['product_id']))

# Get all the user IDs
user_ids = ds_ecommerce.batch(1000000).map(lambda x: x["user_id"])
unique_user_ids = np.unique(np.concatenate(list(user_ids))).astype(str)

# Get all the product IDs
product_ids = ds_ecommerce.batch(1000000).map(lambda x: x["product_id"])
unique_product_ids = np.unique(np.concatenate(list(product_ids))).astype(str)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [5]:
ds_products = tf.data.Dataset.from_tensor_slices(dict(df_products))
ds_products = (
    ds_products.map(lambda x: {
        'product_id': tf.strings.as_string(x['product_id']),
        'product_name': x['product_name'],
        'product_description': x['product_description']
    })
)

# Module 05 - Ranker

In [6]:
class UserModelSearchData(tf.keras.Model):

  def __init__(self, unique_user_id_list, max_tokens=1000, output_sequence_length=30):
    super().__init__()

    # User ID Embedding
    self.user_embedding = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=unique_user_id_list, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_id_list) + 1, 32),
    ])

    # User age
    self.age_normalizer = tf.keras.layers.experimental.preprocessing.Normalization(axis=None)

    # Search Query Embedding
    self.search_vectorization = tf.keras.layers.experimental.preprocessing.TextVectorization(
        max_tokens=max_tokens, output_sequence_length=output_sequence_length)
    self.search_embedding = tf.keras.layers.Embedding(max_tokens, 32, mask_zero=True)

    # Fully-Connected Dense Network
    self.dense = tf.keras.layers.Dense(32)

  def call(self, inputs):
    user_emb = self.user_embedding(inputs['user_id'])
    age = self.age_normalizer(inputs['age'])

    # Incorporate search history
    search_seq = self.search_vectorization(inputs['search_query'])
    search_emb = self.search_embedding(search_seq)
    search_emb = tf.reduce_mean(search_emb, axis=1)

    # Concatenate user features
    concatenated = tf.concat([user_emb, tf.reshape(age, (-1, 1)), search_emb], axis=1)
    return self.dense(concatenated)

  def adapt(self, data):
    age_data = data.map(lambda x: x['age'])
    self.age_normalizer.adapt(age_data)
    search_data = data.map(lambda x: x['search_query'])
    self.search_vectorization.adapt(search_data)

# User and Product models.
class ProductModelMetaData(tf.keras.Model):

  def __init__(self, unique_product_ids, max_tokens=1000, output_sequence_length=30):
    super().__init__()

    # Product ID Embedding
    self.product_embedding = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=unique_product_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_product_ids) + 1, 32)
    ])

    # Product Name Embedding
    self.name_vectorization = tf.keras.layers.experimental.preprocessing.TextVectorization(
        max_tokens=max_tokens, output_sequence_length=output_sequence_length)
    self.name_embedding = tf.keras.layers.Embedding(max_tokens, 32, mask_zero=True)

    # Product Description Embedding
    self.description_vectorization = tf.keras.layers.experimental.preprocessing.TextVectorization(
        max_tokens=max_tokens, output_sequence_length=output_sequence_length)
    self.description_embedding = tf.keras.layers.Embedding(max_tokens, 32, mask_zero=True)

    # Fully-Connected Dense Network
    self.dense = tf.keras.layers.Dense(32)

  def call(self, inputs):
    product_emb = self.product_embedding(inputs['product_id'])

    # Incorporate product description
    name_seq = self.name_vectorization(inputs['product_name'])
    name_emb = self.name_embedding(name_seq)
    name_emb = tf.reduce_mean(name_emb, axis=1)

    # Incorporate product description
    description_seq = self.description_vectorization(inputs['product_description'])
    description_emb = self.description_embedding(description_seq)
    description_emb = tf.reduce_mean(description_emb, axis=1)

    # Concatenate product features
    concatenated = tf.concat([product_emb, name_emb, description_emb], axis=1)
    return self.dense(concatenated)

  def adapt(self, data):
    description = data.map(lambda x: x['product_description'])
    self.description_vectorization.adapt(description)
    name = data.map(lambda x: x['product_name'])
    self.name_vectorization.adapt(name)

# Define the two-tower model.
class TwoTowerModelRanker(tfrs.models.Model):

  def __init__(self, user_model, product_model, task):
    super().__init__()
    self.user_model = user_model
    self.product_model = product_model
    self.task = task

  def compute_loss(self, features, training=False):
    user_embeddings = self.user_model(features)
    positive_product_embeddings = self.product_model(features)
    return self.task(user_embeddings, positive_product_embeddings)

  def predict(self, features, training=False):
    user_embeddings = self.user_model(features)
    product_embeddings = self.product_model(features)
    # Dot product
    dot_products = tf.reduce_sum(user_embeddings * product_embeddings, axis=1)
    # Apply sigmoid transformation
    scores = tf.sigmoid(dot_products)
    return scores

# Instantiate and compile the model.
user_model = UserModelSearchData(unique_user_ids)
user_model.adapt(ds_ecommerce.batch(1000))

product_model = ProductModelMetaData(unique_product_ids)
product_model.adapt(ds_products.batch(1000))

# Calculate embeddings for all products.
product_embeddings = ds_products.batch(100).map(product_model)

# Specify the task.
task = tfrs.tasks.Retrieval(
    metrics=tfrs.metrics.FactorizedTopK(candidates=product_embeddings)
)

model = TwoTowerModelRanker(user_model, product_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

# Train for 3 epochs.
model.fit(ds_ecommerce.batch(1000), epochs=1)

5/5 [==============================] - 5s 204ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0538 - factorized_top_k/top_5_categorical_accuracy: 0.0684 - factorized_top_k/top_10_categorical_accuracy: 0.0772 - factorized_top_k/top_50_categorical_accuracy: 0.1116 - factorized_top_k/top_100_categorical_accuracy: 0.2256 - loss: 6946.8794 - regularization_loss: 0.0000e+00 - total_loss: 6946.8794


In [7]:
# Show prediction
for row in ds_ecommerce.batch(3).take(1):
  print(model.predict(row))

tf.Tensor([0.5238556  0.47241548 0.48426977], shape=(3,), dtype=float32)


# Module 06 - Putting it Together

## ANN Index

In [8]:
scann_retrieval = tfrs.layers.factorized_top_k.ScaNN(
    model.user_model,
    # Number of leaves (clusters)
    num_leaves=100,
    # Top 3 clusters to search from the query to the centroid
    num_leaves_to_search=5,
)

# Add candidates in the index
product_embeddings_to_index = ds_products.batch(1000).map(lambda x: model.product_model(x))
scann_retrieval.index_from_dataset(product_embeddings_to_index)

In [9]:
# Input Data
K = 10

# Get some recommendations.
user_id = tf.constant(["1"])  # user_id should be a string tensor
user_age = tf.constant([25])  # user_age should be a numeric tensor
user_search = tf.constant(['shirt'])
user_query = {"user_id": user_id, "age": user_age, "search_query": user_search}

# Generate candidates
_, candidates = scann_retrieval(user_query, k=K)
candidates = tf.strings.as_string(candidates)

# Generate ranker scores
filtered_candidates = ds_products.filter(lambda x: tf.math.reduce_any(tf.math.equal(x['product_id'], candidates)))

In [10]:
print(candidates)

#tf.math.equal(ds_products['product_id'], candidates)
print(tf.math.reduce_any([True, True, False, False]))
print(list(filtered_candidates.as_numpy_iterator()))

tf.Tensor([[b'199' b'225' b'259' b'354' b'189' b'133' b'184' b'370' b'146' b'11']], shape=(1, 10), dtype=string)
tf.Tensor(True, shape=(), dtype=bool)
[{'product_id': b'11', 'product_name': b'Practical Soft Mouse', 'product_description': b'The Nagasaki Lander is the trademarked name of several series of Nagasaki sport bikes, that started with the 1984 ABC800J'}, {'product_id': b'133', 'product_name': b'Ergonomic Wooden Sausages', 'product_description': b'The slim & simple Maple Gaming Keyboard from Dev Byte comes with a sleek body and 7- Color RGB LED Back-lighting for smart functionality'}, {'product_id': b'146', 'product_name': b'Refined Rubber Table', 'product_description': b'The Apollotech B340 is an affordable wireless mouse with reliable connectivity, 12 months battery life and modern design'}, {'product_id': b'184', 'product_name': b'Practical Wooden Keyboard', 'product_description': b'New ABC 13 9370, 13.3, 5th Gen CoreA5-8250U, 8GB RAM, 256GB SSD, power UHD Graphics, OS 10 Hom

In [11]:
# Convert user_query to a tf.data.Dataset
user_query_ds = tf.data.Dataset.from_tensor_slices(user_query)
#print(list(user_query_ds.as_numpy_iterator()))

# Repeat it the same number of times as the length of filtered_candidates
user_query_ds = user_query_ds.repeat(K)

# Zip the two datasets together
combined_ds = tf.data.Dataset.zip((user_query_ds, filtered_candidates))
print(combined_ds)

# Now merge the data in each pair to make a single dictionary
def merge_fn(user_data, product_data):
    merged_data = user_data.copy()
    merged_data.update(product_data)
    return merged_data

merged_ds = combined_ds.map(merge_fn)
print(merged_ds)

# Check the result
result = []
for item in merged_ds.batch(5):
  _id = item['product_id'].numpy()
  name = item['product_name'].numpy()
  score = model.predict(item).numpy()
  batch_result = [(i, n, s) for i, n, s in zip(_id, name, score)]
  result += batch_result
result = sorted(result, key=lambda x: x[2], reverse=True)
result

<ZipDataset element_spec=({'user_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'age': TensorSpec(shape=(), dtype=tf.int32, name=None), 'search_query': TensorSpec(shape=(), dtype=tf.string, name=None)}, {'product_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'product_name': TensorSpec(shape=(), dtype=tf.string, name=None), 'product_description': TensorSpec(shape=(), dtype=tf.string, name=None)})>
<MapDataset element_spec={'user_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'age': TensorSpec(shape=(), dtype=tf.int32, name=None), 'search_query': TensorSpec(shape=(), dtype=tf.string, name=None), 'product_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'product_name': TensorSpec(shape=(), dtype=tf.string, name=None), 'product_description': TensorSpec(shape=(), dtype=tf.string, name=None)}>


[(b'259', b'Rustic Wooden Hat', 0.5912493),
 (b'184', b'Practical Wooden Keyboard', 0.54599226),
 (b'354', b'Handcrafted Metal Car', 0.53106886),
 (b'189', b'Practical Fresh Fish', 0.524371),
 (b'11', b'Practical Soft Mouse', 0.49213916),
 (b'370', b'Elegant Wooden Bike', 0.48564225),
 (b'146', b'Refined Rubber Table', 0.48314574),
 (b'133', b'Ergonomic Wooden Sausages', 0.4660032),
 (b'225', b'Generic Frozen Chicken', 0.4659229),
 (b'199', b'Elegant Granite Chicken', 0.4593957)]

# Module 07: GCP Deployment


**GCP set up**

1.   Create a GCP account and activate the free tiers
2.   Create a project named **mlemasterclass-deployment**
1.   Create a new Cloud Storage bucket, titled **mlemasterclass-deployment-bucket** and specify the region **us-central1**
1.   Make sure that the bucket is publicly accessed
2.   Enable VertexAI APIs
1.   Enable Compute Engine API









**Declare the first GCP variables**

In [12]:
PROJECT_ID = "apt-honor-390612"
REGION = "us-central1"

MODEL_ARTIFACT_DIR = "prediction-model"

**Make sure the Cloud SDK uses the right project for all the commands in this notebook.**

In [13]:
!gcloud config set project $PROJECT_ID

Updated property [core/project].


**Authenticate your Google Cloud account**

This provides access to your Cloud Storage bucket and lets you submit training jobs and prediction requests.

In [14]:
import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

**Declare the Bucket variables**

In [15]:
BUCKET_NAME = "mlemasterclass-deployment-bucket"
BUCKET_URI = f"gs://{BUCKET_NAME}"

**Show the stored folders/files**

In [16]:
!gsutil ls -al $BUCKET_URI

                                 gs://mlemasterclass-deployment-bucket/prediction-model/


Upload the ScaNN model to the Bucket

Before you can deploy your model for serving, Vertex AI needs access to the following files in Cloud Storage:

In [17]:
%mkdir app

In [18]:
tf.saved_model.save(scann_retrieval, "./app/index")

In [19]:
!gsutil cp -r ./app/index/* {BUCKET_URI}/{MODEL_ARTIFACT_DIR}/

Copying file://./app/index/fingerprint.pb [Content-Type=application/octet-stream]...
Copying file://./app/index/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://./app/index/variables/variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
Copying file://./app/index/variables/variables.index [Content-Type=application/octet-stream]...
| [4 files][721.3 KiB/721.3 KiB]                                                
Operation completed over 4 objects/721.3 KiB.                                    


In [20]:
!gsutil ls -al $BUCKET_URI

                                 gs://mlemasterclass-deployment-bucket/prediction-model/


Test the loaded model

In [22]:
loaded  = tf.saved_model.load(f'{BUCKET_URI}/{MODEL_ARTIFACT_DIR}')
trash, candidates = loaded(user_query)
print(candidates)

tf.Tensor([[199 225 259 354 189 133 184 370 146  11]], shape=(1, 10), dtype=int32)


**Initialize Vertex AI SDK for Python**

In [23]:
from google.cloud import aiplatform
aiplatform.init(project=PROJECT_ID, location=REGION)

**Declare the required variables**

In [24]:
REPOSITORY = "prediction-model-container"
IMAGE = "model-api"
MODEL_DISPLAY_NAME = "deployment-container-api"

**Create the FastAPI code**

To serve predictions from the classification model, build a FastAPI server application.

In [25]:
%%writefile app/main.py
from fastapi import FastAPI, Request

import os
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

from google.cloud import storage

def json_to_tf(dictionnary):

    transformed_dict = {}

    for key, value in dictionnary.items():
        transformed_dict[key] = tf.constant(value)

    return(transformed_dict)

app = FastAPI()

scann_index = tf.saved_model.load(f'{os.environ["BUCKET_URI"]}/{os.environ["MODEL_ARTIFACT_DIR"]}')

@app.get("/hello")
def greet_user():
    return {"message": "Hello! Welcome to MLE MasterClass!"}

@app.get(os.environ["AIP_HEALTH_ROUTE"], status_code=200)
def health():
    return {"message": "Model API is healthy!"}

@app.post(os.environ["AIP_PREDICT_ROUTE"])
async def predict(request: Request):
    body = await request.json()
    instances = body["instances"]
    trash, outputs = scann_index(json_to_tf(instances[0]))
    outputs = [arr.tolist() for arr in outputs.numpy()]

    return {"predictions": [class_num for class_num in outputs]}

Writing app/main.py


**Add pre-start script**

FastAPI executes the following script before starting up the server. The PORT environment variable is set to equal to AIP_HTTP_PORT in order to run FastAPI on the same port expected by Vertex AI.

In [26]:
%%writefile app/prestart.sh
#!/bin/bash
export PORT=$AIP_HTTP_PORT

Writing app/prestart.sh


**Create the test instances.json that store some data that will be used for prediction**

In [27]:
%%writefile instances.json
{
    "instances": [
        {"user_id": ["1", "2"], "age": [25, 45], "search_query": ["shirt", "long"]}
    ]
}

Writing instances.json


**Create the requirements.txt file that will be used by the Docker Image**

In [28]:
%%writefile requirements.txt
google-cloud-storage>=1.26.0,<2.0.0dev
google-cloud-aiplatform
shapely<2
tensorflow==2.11.1
protobuf==3.19.6
tensorflow-recommenders==0.7.3
tensorflow-datasets==3.2.0
tensorflow-metadata==0.22.2
scann
dill==0.3.6

Writing requirements.txt


Write the Dockerfile, using tiangolo/uvicorn-gunicorn-fastapi as a base image. This automatically runs FastAPI for you using Gunicorn and Uvicorn.

Visit [https://fastapi.tiangolo.com/deployment/docker/](https://fastapi.tiangolo.com/deployment/docker/) to learn more.

In [29]:

%%writefile Dockerfile

FROM tiangolo/uvicorn-gunicorn-fastapi:python3.9

COPY ./app /app
COPY requirements.txt requirements.txt

RUN pip install -r requirements.txt

EXPOSE 8080

ENV BUCKET_URI=gs://mlemasterclass-deployment-bucket
ENV MODEL_ARTIFACT_DIR=prediction-model
ENV AIP_HTTP_PORT=8080
ENV AIP_HEALTH_ROUTE=health
ENV AIP_PREDICT_ROUTE=predict

Writing Dockerfile


**Push the container to artifact registry**

Create your repository in the Artifact registry and push your container image to the repository. Run this below cell once to create the artifact repository.

In [30]:
!gcloud artifacts repositories create {REPOSITORY} \
    --repository-format=docker \
    --location=$REGION

ERROR: (gcloud.artifacts.repositories.create) ALREADY_EXISTS: the repository already exists


Push the image to the created artifact repository using Cloud-Build. (3~5 min)

Note: The following command automatically considers the Dockerfile from the directory it is being run from.

In [31]:
!gcloud builds submit --region={REGION} --tag={REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}

Creating temporary tarball archive of 35 file(s) totalling 56.1 MiB before compression.
Uploading tarball of [.] to [gs://apt-honor-390612_cloudbuild/source/1687555310.795324-df5dca0f760c44bdb59e2c2504415592.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/apt-honor-390612/locations/us-central1/builds/b0bdd2e1-d3e0-4b07-af21-8cd595812b9d].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds;region=us-central1/b0bdd2e1-d3e0-4b07-af21-8cd595812b9d?project=637218062560 ].
 REMOTE BUILD OUTPUT
starting build "b0bdd2e1-d3e0-4b07-af21-8cd595812b9d"

FETCHSOURCE
Fetching storage object: gs://apt-honor-390612_cloudbuild/source/1687555310.795324-df5dca0f760c44bdb59e2c2504415592.tgz#1687555322834225
Copying gs://apt-honor-390612_cloudbuild/source/1687555310.795324-df5dca0f760c44bdb59e2c2504415592.tgz#1687555322834225...
/ [1 files][  7.0 MiB/  7.0 MiB]                                                
Operation completed over 1 objects/7.0 MiB.
BUILD
Already have

# **Deploy to Vertex AI**

**Create Vertex AI model using artifact uri**

Use the Python SDK to upload and deploy your model from the artifact registry. (2~3 min)

In [32]:
#from google.cloud import aiplatform
#aiplatform.init(project=PROJECT_ID, location=REGION)

model = aiplatform.Model.upload(display_name=MODEL_DISPLAY_NAME,
                                artifact_uri=f"{BUCKET_URI}/{MODEL_ARTIFACT_DIR}",
                                serving_container_image_uri=f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}")

Creating Model


INFO:google.cloud.aiplatform.models:Creating Model


Create Model backing LRO: projects/637218062560/locations/us-central1/models/5153131723432656896/operations/4463811806254596096


INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/637218062560/locations/us-central1/models/5153131723432656896/operations/4463811806254596096


Model created. Resource name: projects/637218062560/locations/us-central1/models/5153131723432656896@1


INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/637218062560/locations/us-central1/models/5153131723432656896@1


To use this Model in another session:


INFO:google.cloud.aiplatform.models:To use this Model in another session:


model = aiplatform.Model('projects/637218062560/locations/us-central1/models/5153131723432656896@1')


INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/637218062560/locations/us-central1/models/5153131723432656896@1')


**Deploy the model to Vertex AI Endpoints**

Deploy the model to a Vertex AI Endpoint. After this step completes, the model is deployed and ready for online predictions. (~5-10 min)

In [33]:
endpoint = model.deploy(machine_type="n1-standard-4")

Creating Endpoint


INFO:google.cloud.aiplatform.models:Creating Endpoint


Create Endpoint backing LRO: projects/637218062560/locations/us-central1/endpoints/3878122646700818432/operations/8033477460899135488


INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/637218062560/locations/us-central1/endpoints/3878122646700818432/operations/8033477460899135488


Endpoint created. Resource name: projects/637218062560/locations/us-central1/endpoints/3878122646700818432


...

INFO:google.cloud.aiplatform.models:Deploying model to Endpoint : projects/637218062560/locations/us-central1/endpoints/3878122646700818432


Deploy Endpoint model backing LRO: projects/637218062560/locations/us-central1/endpoints/3878122646700818432/operations/424082940503261184


INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/637218062560/locations/us-central1/endpoints/3878122646700818432/operations/424082940503261184


Endpoint model deployed. Resource name: projects/637218062560/locations/us-central1/endpoints/3878122646700818432


INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/637218062560/locations/us-central1/endpoints/3878122646700818432


**Request predictions**

Send online requests to the model deployed to the endpoint and get predictions.

**Using Python SDK**

Get predictions from the endpoint for a sample input using python SDK.

In [34]:
import json

# Load JSON from a file
with open('instances.json', 'r') as file:
    data = json.load(file)

# Access the loaded data
print(data)

{'instances': [{'user_id': ['1', '2'], 'age': [25, 45], 'search_query': ['shirt', 'long']}]}


In [35]:
endpoint.predict(instances= data['instances'])

Prediction(predictions=[[199.0, 225.0, 259.0, 354.0, 189.0, 133.0, 184.0, 370.0, 146.0, 11.0], [225.0, 214.0, 242.0, 252.0, 11.0, 146.0, 370.0, 264.0, 401.0, 258.0]], deployed_model_id='8073355441968513024', model_version_id='1', model_resource_name='projects/637218062560/locations/us-central1/models/5153131723432656896', explanations=None)

**Using REST**

Get predictions from the endpoint using curl request.

In [36]:
ENDPOINT_ID = endpoint.name
print("ENDPOINT_ID :", ENDPOINT_ID)

ENDPOINT_ID : 3878122646700818432


In [37]:
! curl -X POST \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
-d @instances.json \
https://{REGION}-aiplatform.googleapis.com/v1/projects/{PROJECT_ID}/locations/{REGION}/endpoints/{ENDPOINT_ID}:predict

{
  "predictions": [
    [
      199,
      225,
      259,
      354,
      189,
      133,
      184,
      370,
      146,
      11
    ],
    [
      225,
      214,
      242,
      252,
      11,
      146,
      370,
      264,
      401,
      258
    ]
  ],
  "deployedModelId": "8073355441968513024",
  "model": "projects/637218062560/locations/us-central1/models/5153131723432656896",
  "modelDisplayName": "deployment-container-api",
  "modelVersionId": "1"
}


# **Cleaning up**

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) project you used for the tutorial.

# **Exercice**

1.   Try to deploy the Docker Container on localhost using docker build, docker run commands (dont forget to set the env variables required by the FastAPI (AIP_PREDICT_ROUTE, etc...)
2.   Try to add the ranker module to the application and deploy the new application locally and on GCP

